# OCR on lined paper with shadow removal

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import pytesseract
import os
import subprocess

# Path to tesseract executable (in case it isn't in your PATH)
try:
    subprocess.call(["tesseract"])
except FileNotFoundError:
    pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

PARENT_DIR = os.path.dirname(os.path.dirname(os.path.realpath("FILEPATH")))
image_name = "005.jpg"
image = cv2.imread(os.path.join(PARENT_DIR, "images", image_name), cv2.IMREAD_COLOR)

# Check if image is loaded fine
if image is None:
    print('Error opening image')

# Tesseract OCR before processing
text = pytesseract.image_to_string(image)
print("Before processing:\n" + "\033[92m{}\033[00m".format(text))
plt.imshow(image)

In [ ]:
# # check if there are lines in the image

# PROBABILISTIC = True

# gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
# edges = cv2.Canny(gray, 50, 150, apertureSize=3)
# lines_on_image = image.copy()

# if PROBABILISTIC:
#     lines = cv2.HoughLinesP(edges,1,np.pi/180,100,minLineLength=100,maxLineGap=10)
#     if lines is not None:
#         print("Lines detected")
#         # check if the lines are a considerable number (e.g. 5)
#         if len(lines) > 5:
#             print(len(lines))
#             # check if the lines are long enough (e.g. one fourth of the image width)
#             long_enough_lines = 0
#             for line in lines:
#                 x1, y1, x2, y2 = line[0]
#                 cv2.line(lines_on_image, (x1, y1), (x2, y2), (0, 0, 255), 2)
#                 if abs(x2 - x1) > image.shape[1] / 5:
#                     long_enough_lines += 1
#             print("Long enough lines:",long_enough_lines)
#             if long_enough_lines > 5:
#                 print("The text is on lined paper")
#     else:
#         print("No lines detected")
# else:
#     lines = cv2.HoughLines(edges, 1, np.pi / 180, 200)
#     if lines is not None:
#         print("Lines detected")    
#         # check if the lines are a considerable number (e.g. 5)
#         if len(lines) > 5:
#             print(len(lines))
#             # check if the lines are long enough (e.g. one third of the image width)
#             long_enough_lines = 0
#             for rho, theta in lines[:, 0]:
#                 a = np.cos(theta)
#                 b = np.sin(theta)
#                 x0 = a * rho
#                 y0 = b * rho
#                 x1 = int(x0 + 1000 * (-b))
#                 y1 = int(y0 + 1000 * (a))
#                 x2 = int(x0 - 1000 * (-b))
#                 y2 = int(y0 - 1000 * (a))
#                 cv2.line(lines_on_image, (x1, y1), (x2, y2), (0, 0, 255), 2)
#                 if abs(x2 - x1) > image.shape[1] / 3:
#                     long_enough_lines += 1
#             print("Long enough lines:", long_enough_lines)
#             if long_enough_lines > 5:
#                 print("The text is on lined paper")
#     else:
#         print("No lines detected")

# # show the image with lines
# plt.imshow(lines_on_image)

In [ ]:
# image = cv2.compareHist(image)
img = np.copy(image)

# Shadow removal of the image
rgb_planes = cv2.split(img)

result_planes = []
result_norm_planes = []
for plane in rgb_planes:
    dilated_img = cv2.dilate(plane, np.ones((7, 7), np.uint8))
    bg_img = cv2.medianBlur(dilated_img, 21)
    diff_img = 255 - cv2.absdiff(plane, bg_img)
    norm_img = cv2.normalize(diff_img, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8UC1)
    result_planes.append(diff_img)
    result_norm_planes.append(norm_img)

result = cv2.merge(result_planes)
result_norm = cv2.merge(result_norm_planes)

# Show source image
plt.imshow(result_norm, cmap="gray")

In [ ]:
# Apply adaptiveThreshold
result_norm = cv2.cvtColor(result_norm, cv2.COLOR_BGR2GRAY)
adaptive = cv2.adaptiveThreshold(result_norm, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 11, 2)
otsu = bw = cv2.threshold(result_norm, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

bw = cv2.morphologyEx(bw, cv2.MORPH_OPEN, np.ones((3, 3), np.uint8))
bw = cv2.bitwise_not(bw)

# Tesseract OCR after processing
text = pytesseract.image_to_string(bw)
print("Tesseract OCR:\n" + "\033[92m{}\033[00m".format(text))

# Show difference between adaptive and otsu thresholding
plt.subplot(121), plt.imshow(adaptive, 'gray'), plt.title('adaptive')
plt.xticks([]), plt.yticks([])
plt.subplot(122), plt.imshow(otsu, 'gray'), plt.title('otsu')
plt.xticks([]), plt.yticks([])
plt.show()

In [ ]:
# Create the images that will use to extract the horizontal and vertical lines
horizontal = np.copy(bw)

# Specify size on horizontal axis
cols = horizontal.shape[1]
horizontal_size = cols // 30

# Create structure element for extracting horizontal lines through morphology operations
horizontalStructure = cv2.getStructuringElement(cv2.MORPH_RECT, (horizontal_size, 1))

# Apply morphology operations
horizontal = cv2.morphologyEx(bw, cv2.MORPH_OPEN, horizontalStructure, iterations=2)

horizontal = cv2.dilate(horizontal, np.ones((3, 3)))
horizontal = cv2.bitwise_not(horizontal)
# Show extracted grid
plt.imshow(horizontal, cmap='gray')

In [ ]:
# check if there are lines in the image

PROBABILISTIC = True

# gray = cv2.cvtColor(horizontal, cv2.COLOR_BGR2GRAY)
edges = cv2.Canny(horizontal, 50, 150, apertureSize=3)
lines_on_image = image.copy()

if PROBABILISTIC:
    lines = cv2.HoughLinesP(edges,1,np.pi/180,100,minLineLength=100,maxLineGap=10)
    if lines is not None:
        print("Lines detected")
        # check if the lines are a considerable number (e.g. 5)
        if len(lines) > 5:
            print(len(lines))
            # check if the lines are long enough (e.g. one fourth of the image width)
            long_enough_lines = 0
            for line in lines:
                x1, y1, x2, y2 = line[0]
                cv2.line(lines_on_image, (x1, y1), (x2, y2), (0, 0, 255), 2)
                if abs(x2 - x1) > image.shape[1] / 5:
                    long_enough_lines += 1
            print("Long enough lines:",long_enough_lines)
            if long_enough_lines > 5:
                print("The text is on lined paper")
    else:
        print("No lines detected")
else:
    lines = cv2.HoughLines(edges, 1, np.pi / 180, 200)
    if lines is not None:
        print("Lines detected")    
        # check if the lines are a considerable number (e.g. 5)
        if len(lines) > 5:
            print(len(lines))
            # check if the lines are long enough (e.g. one fourth of the image width)
            long_enough_lines = 0
            for rho, theta in lines[:, 0]:
                a = np.cos(theta)
                b = np.sin(theta)
                x0 = a * rho
                y0 = b * rho
                x1 = int(x0 + 1000 * (-b))
                y1 = int(y0 + 1000 * (a))
                x2 = int(x0 - 1000 * (-b))
                y2 = int(y0 - 1000 * (a))
                cv2.line(lines_on_image, (x1, y1), (x2, y2), (0, 0, 255), 2)
                if abs(x2 - x1) > image.shape[1] / 4:
                    long_enough_lines += 1
            print("Long enough lines:", long_enough_lines)
            if long_enough_lines > 5:
                print("The text is on lined paper")
    else:
        print("No lines detected")

# show the image with lines
plt.imshow(lines_on_image)

In [ ]:
remove_grid = cv2.bitwise_not(cv2.bitwise_and(bw, horizontal))
text = pytesseract.image_to_string(remove_grid)
print("Without opening:\n" + "\033[92m{}\033[00m".format(text))

plt.rcParams['figure.figsize'] = [16, 10]

plt.subplot(121), plt.imshow(remove_grid, cmap='gray'), plt.title('Only removed grid')
plt.xticks([]), plt.yticks([])
# Doing some opening/closing to remove some major noise
opening = cv2.morphologyEx(remove_grid, cv2.MORPH_OPEN, np.ones((5, 5)), iterations=2)
closing = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, np.ones((3, 3)))
text = pytesseract.image_to_string(closing)
print("With opening and closing:\n" + "\033[92m{}\033[00m".format(text))

plt.subplot(122), plt.imshow(closing, cmap='gray'), plt.title('With opening/closing added')
plt.xticks([]), plt.yticks([])
plt.show()